In [1]:
from libs.SatelliteEnvironment import *
from libs.DRLAgents.DQNAgent import *
from libs.Constants import *

import numpy as np
import torch 

C:\Users\minstone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
env = CircularOrbit()

env.reset()

state = env.get_state()

num_states = state.shape[0]
orbit_num  = env.orbit_num
sat_num    = env.satellite_num
num_action = orbit_num * sat_num

print("ORBIT NUM : ", orbit_num)
print("SAT NUM : ", sat_num)
print("STATE NUM : ", num_states)
print("ACTION NUM : ", num_action)

agent      = DQNAgent(num_states = num_states,
                      num_action = num_action)

agent.train()

losses = []
rewards = []

episodes = 10000
for episode in range(episodes):
    env.reset()

    ep_reward = 0
    ep_loss = 0
    step = 0
    while True:
        step += 1
        state = env.get_state()

        action = agent.get_action(state)

        reward, done, info = env.step(action)

        next_state = env.get_state() # 
            
        agent.append_sample(state=state, 
                            action=action, 
                            reward=reward, 
                            next_state=next_state,
                            done=done) # 
        
        ep_reward += reward

        if step >= MEMORY_CAPACITY:

            loss = agent.train_model()
            ep_loss += loss
            
            if(step % 100 == 0):
                agent.update_target()
            
        if done:
            agent.write_summary(ep_reward, ep_loss, 0, step)
            print("episode: {} , the episode reward is {}".format(step, ep_reward))


ORBIT NUM :  2
SAT NUM :  8
STATE NUM :  36
ACTION NUM :  16
episode: 2 , the episode reward is 0.0
episode: 6 , the episode reward is 0.0
episode: 9 , the episode reward is 0.0
episode: 10 , the episode reward is 0.8694577061633916
episode: 11 , the episode reward is -99.13054229383661
episode: 16 , the episode reward is -99.13054229383661
episode: 17 , the episode reward is -98.26108458767322
episode: 19 , the episode reward is -98.26108458767322
episode: 20 , the episode reward is -97.39162688150984
episode: 22 , the episode reward is -197.39162688150984
episode: 29 , the episode reward is -297.39162688150986
episode: 30 , the episode reward is -397.39162688150986
episode: 33 , the episode reward is -397.39162688150986
episode: 36 , the episode reward is -396.5249168913247
episode: 54 , the episode reward is -496.5249168913247
episode: 61 , the episode reward is -596.5249168913247
episode: 79 , the episode reward is -594.788749194976
episode: 82 , the episode reward is -594.78874919

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.